In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from bart_playground import *
import bartz

In [3]:
proposal_probs = {"grow" : 0.4,
                  "prune" : 0.4,
                  "change" : 0.1,
                  "swap" : 0.1}
generator = DataGenerator(n_samples=1000, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
np.set_printoptions(suppress=True)
print(y_train[:12])

[ 0.59965549  0.66000473  0.60195109  0.59759841 -0.48024437 -0.05465378
 -0.09076175 -0.06775115  0.44628062  0.51643207 -0.17564767 -0.18611937]


In [4]:
# initialize numba
bart = DefaultBART(ndpost=200, nskip=100, n_trees=100, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)

Iterations: 100%|██████████| 300/300 [00:09<00:00, 33.00it/s]


In [5]:
bart2 = DefaultBART(ndpost=400, nskip=100, n_trees=100, proposal_probs=proposal_probs)
%prun -s cumtime -D profile_bart.prof -q bart2.fit(X_train, y_train)
!gprof2dot -f pstats profile_bart.prof -o profile_bart.dot
!dot -Tpng profile_bart.dot -o profile_bart.png

Iterations: 100%|██████████| 500/500 [00:09<00:00, 53.27it/s]


 
*** Profile stats marshalled to file 'profile_bart.prof'.


In [6]:
arrays = [tree.vars for tree in bart.trace[-1].trees]
counts = np.array([np.count_nonzero(arr >= 0) for arr in arrays])
print(counts)
deep_trees = np.array([count >= 3 for count in counts])
print(np.where(deep_trees))

[1 1 1 4 3 2 2 1 1 2 1 1 3 2 1 1 5 5 2 1 2 0 1 1 2 1 2 2 2 1 2 1 2 1 0 1 0
 1 1 1 1 1 1 1 1 2 2 1 2 1 2 1 2 2 2 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 2 2 4 2
 1 1 3 1 1 1 1 2 5 1 1 1 2 2 1 2 2 1 1 1 1 2 1 1 0 1]
(array([ 3,  4, 12, 16, 17, 72, 76, 82]),)


In [7]:
print(bart.sampler.trace[-1].global_params)

{'eps_sigma2': array([0.00661782])}


In [8]:
from bart_playground import visualize_tree
tree_sp : Tree = bart.sampler.trace[-1].trees[90]

print(tree_sp)
print(tree_sp.vars)
print(tree_sp.leaf_vals)

X_0 <= 0.650110245 (split, n = 750)
	Val: 0.004659622 (leaf, n = 480)
	X_1 <= 0.315988928 (split, n = 270)
		Val: -0.005203606 (leaf, n = 74)
		Val: -0.028871579 (leaf, n = 196)
[ 0 -1  1 -2 -2 -1 -1 -2]
[        nan  0.00465962         nan         nan         nan -0.00520361
 -0.02887158         nan]


In [9]:
rf = RandomForestRegressor(random_state=42)
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=200, nskip=100)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

INFO:2025-06-23 12:00:55,884:jax._src.xla_bridge:867: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


Iteration 100/300 P_grow=0.57 P_prune=0.43 A_grow=0.28 A_prune=0.40 (burnin)
Iteration 200/300 P_grow=0.54 P_prune=0.46 A_grow=0.30 A_prune=0.24
Iteration 300/300 P_grow=0.52 P_prune=0.48 A_grow=0.25 A_prune=0.31


In [10]:
models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 0.01699852984056321,
 'rf': 0.01570244066836391,
 'lr': 0.04058944845865094,
 'btz': 0.01451765109911874}

In [11]:
print(bart.sampler.trace[-1].evaluate(X_train)[:12])
print(bart.preprocessor.transform_y(y_train)[:12])

[ 0.24605564  0.27110088  0.28510376  0.26839188 -0.33688342  0.09359006
  0.0220605   0.02451161  0.23247432  0.30771708  0.03145187  0.07369719]
[ 0.36207604  0.4024236   0.3636108   0.36070073 -0.35991025 -0.07537408
 -0.09951471 -0.08413056  0.25953454  0.30643553 -0.1562667  -0.16326775]


In [12]:
mean_squared_error(y_test, np.ones_like(y_test) * y_test.mean())

0.08625514042334204

In [13]:
if all([(bart.sampler.trace[-1].trees[i].evaluate() == bart.sampler.trace[-1].trees[i].evaluate(X_train)).all()
            for i in range(100)]):
    print("True")
else:
    print("False")

True


In [14]:
if np.allclose(bart.trace[-1].evaluate(), bart.trace[-1].evaluate(X_train), atol=1e-6):
    print("True")
else:
    print("False")

True
